In [1]:
%cd ..
import os
import time

/home/yongming/apperception


In [2]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F
from apperception.predicate import camera, objects
from optimized_ingestion.utils.preprocess import preprocess
database.connection
from optimized_ingestion.cache import disable_cache
disable_cache()

CUDA is available.
 > 0: Tesla T4


In [3]:
NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"
if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
NUSCENES_RAW_DATA = "NUSCENES_RAW_DATA"
if NUSCENES_RAW_DATA in os.environ:
    RAW_DATA_DIR = os.environ[NUSCENES_RAW_DATA]
else:
    RAW_DATA_DIR = "/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Mini"

In [4]:
import cv2
from os.path import exists
def construct_video(frames, size=(1600, 900), base=False, vid_name=None, vid_prefix=False):
    unique_frames = []
    for f in frames:
        if f not in unique_frames:
            unique_frames.append(f)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vid_name = "./outputs/fig13_"+vid_name if vid_name else f"./outputs/{'fig13_base' if base else 'fig13_optimized'}.mp4"
    display_video = cv2.VideoWriter(vid_name,fourcc, 1, size)
    for frame in unique_frames:
        if vid_prefix:
            img_path = os.path.join(RAW_DATA_DIR, 'sweeps/CAM_FRONT', frame)
            if not exists(img_path):
                img_path = os.path.join(RAW_DATA_DIR, 'samples/CAM_FRONT', frame)
        else:
            img_path = os.path.join(RAW_DATA_DIR, frame)
        img = cv2.imread(img_path)
        display_video.write(img)

    display_video.release()

In [5]:
name = 'ScenicWorld'
world = empty_world(name=name)

obj1 = objects[0]
obj2 = objects[1]
cam = camera

# car_world = empty_world(name=name).filter(
#     (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck') | F.like(obj1.type, 'bus'))
# )
# world = world.filter(
#     (obj1.id != obj2.id) &
#     (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck')) &
#     (F.like(obj2.type, 'car') | F.like(obj2.type, 'truck')) &
#     F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego)), -15, 15) &
#     (F.distance(cam.ego, obj1.trans@cam.time) < 50) &
#     (F.view_angle(obj1.trans@cam.time, cam.ego) < 70 / 2.0) &
#     (F.distance(cam.ego, obj2.trans@cam.time) < 50) &
#     (F.view_angle(obj2.trans@cam.time, cam.ego) < 70 / 2.0) &
#     F.contains_all('intersection', [obj1.trans, obj2.trans]@cam.time) &
#     F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.ego), 40, 135) &
#     F.angle_between(F.facing_relative(obj2.trans@cam.time, cam.ego), -135, -50) &
#     (F.min_distance(cam.ego, 'intersection') < 10) &
#     F.angle_between(F.facing_relative(obj1.trans@cam.time, obj2.trans@cam.time), 100, -100)
# )
world = world.filter(
    (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck')) &
    F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego)), -15, 15) &
    (F.distance(cam.ego, obj1.trans@cam.time) < 50) &
    F.contains_all('intersection', [obj1.trans]@cam.time) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.ego), 135, 225) &
    (F.min_distance(cam.ego, 'intersection') < 10)
)


In [6]:
preprocess(world, DATA_DIR, ['scene-0655-CAM_FRONT'], base=False)

scene-0655-CAM_FRONT --------------------------------------------------------------------------------
Stage:  InView
Stage:  DecodeFrame.ParallelDecodeFrame


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.24s/it]


Stage:  Detection2D.YoloDetection


ego_speed: 8.223887668036209
next frame name None
next frame num 0
Action action type: exit_view,
        start time: 2018-08-27 15:51:32.112404,
        finish time: 2018-08-27 15:51:32.112404,
        start loc: (1899.1487444481882, 874.5602868735198, 0.0),
        end loc: (1772.143798828125, 866.3028564453125, 0.0)
        estimated time: 0:00:00


Stage:  Detection2D.ObjectTypeFilter
Stage:  Detection3D.FromDetection2DAndRoad
Stage:  DetectionEstimation


next frame name None
next frame num 2
Action action type: car_exit_segment,
        start time: 2018-08-27 15:51:32.162404,
        finish time: 2018-08-27 15:51:32.241057,
        start loc: (1894.2887226660048, 874.5591050971786, 0.0),
        end loc: (1894.991933355613, 874.562164556387)
        estimated time: 0:00:00.078653
next frame name None
next frame num 5
Action action type: car_exit_segment,
        start time: 2018-08-27 15:51:32.262404,
        finish time: 2018-08-27 15:51:32.498523,
        start loc: (1808.6396827961112, 874.9212010546997, -2.220446049250313e-16),
        end loc: (1810.132452086616, 876.4139702359263)
        estimated time: 0:00:00.236119
next frame name None
next frame num 6
Action action type: car_exit_segment,
        start time: 2018-08-27 15:51:32.512404,
        finish time: 2018-08-27 15:51:32.606565,
        start loc: (1903.880429084013, 875.3859606571668, 0.0),
        end loc: (1904.4757275161733, 875.9812590457484)
        estimated time

next frame name None
next frame num 41
Action action type: car_exit_segment,
        start time: 2018-08-27 15:51:35.262404,
        finish time: 2018-08-27 15:51:35.497359,
        start loc: (1815.0915245356862, 874.4113798738849, 0.0),
        end loc: (1816.5769340016054, 875.8967892310645)
        estimated time: 0:00:00.234955
next frame name None
next frame num 44
Action action type: car_exit_segment,
        start time: 2018-08-27 15:51:35.512404,
        finish time: 2018-08-27 15:51:35.720029,
        start loc: (1815.1790835762959, 874.4549736187295, 0.0),
        end loc: (1816.4917085242153, 875.7675984705583)
        estimated time: 0:00:00.207625
next frame name None
next frame num 46
Action action type: car_exit_segment,
        start time: 2018-08-27 15:51:35.762404,
        finish time: 2018-08-27 15:51:35.960078,
        start loc: (1815.5223754656404, 873.9447490596755, 0.0),
        end loc: (1816.7720887999537, 875.1944623025037)
        estimated time: 0:00:00.19

next frame name None
next frame num 148
Action action type: car_exit_segment,
        start time: 2018-08-27 15:51:44.362404,
        finish time: 2018-08-27 15:51:44.521478,
        start loc: (1986.5214489270088, 862.6055351369258, 0.0),
        end loc: (1987.7254659801713, 861.8484736767291)
        estimated time: 0:00:00.159074
next frame name None
next frame num 150
Action action type: car_exit_segment,
        start time: 2018-08-27 15:51:44.612404,
        finish time: 2018-08-27 15:51:44.676737,
        start loc: (1972.895412352329, 831.0638107857424, 0.0),
        end loc: (1973.2296170483828, 831.5319474172555)
        estimated time: 0:00:00.064333
next frame name None
next frame num 173
Action action type: car_exit_segment,
        start time: 2018-08-27 15:51:46.012404,
        finish time: 2018-08-27 15:51:46.618658,
        start loc: (2044.1446464605508, 877.3061032986698, 0.0),
        end loc: (2047.9774473750654, 881.1389039326042)
        estimated time: 0:00:00.

check_detections [[0, 728, 463, 785, 509, 'lanegroup', (array('d', [632.5673278607485, 595.0064038929324, 723.9896110386784, 739.2077568337414, 756.3507025947408, 814.9641409917908, 770.6435112678597, 632.5673278607485]), array('d', [528.9128096702734, 528.3205624725372, 501.6543160656389, 498.422248120345, 498.66987355238507, 499.49252554956377, 530.8904272699301, 528.9128096702734])), 'sweeps/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385109512404.jpg']]
Stage:  Tracking2D.StrongSORT
Stage:  Tracking3D.FromTracking2DAndRoad
Stage:  SegmentTrajectory.FromTracking3D
Time taken to insert trajectories: 0.31035542488098145
num_video:  1
total preprocess time 23.398338794708252


In [7]:
import numpy as np
investigation =[[0, 0, 479, 269, 630, None, (1800.8542502977525, 868.6106570326535, 0.0), ((1800.8319844478135, 868.9208206918607, 0.0), (1800.8765161476917, 868.3004933734464, 0.0))], [1, 236, 464, 429, 578, None, (1801.2291096786573, 868.305008940877, 0.0), ((1801.2102934815034, 868.5555922250609, 0.0), (1801.2479258758112, 868.0544256566931, 0.0))], [2, 403, 465, 527, 552, None, (1801.405934482298, 868.011015809015, 0.0), ((1801.3929661328764, 868.1804623520046, 0.0), (1801.4189028317198, 867.8415692660254, 0.0))], [3, 603, 473, 658, 522, None, (1801.5876508371173, 867.586681546114, 0.0), ((1801.5814949471257, 867.6656835716167, 0.0), (1801.5938067271088, 867.5076795206114, 0.0))], [4, 1204, 459, 1445, 573, None, (1801.4905352468109, 865.643078417144, 0.0), ((1801.465217818611, 865.977737334164, 0.0), (1801.5158526750108, 865.308419500124, 0.0))], [5, 730, 481, 780, 514, None, (1801.6156124213599, 867.2278346865218, 0.0), ((1801.6099787652952, 867.3001345880323, 0.0), (1801.6212460774245, 867.1555347850112, 0.0))], [6, 1114, 467, 1238, 543, None, (1801.6050544133154, 866.0208889300488, 0.0), ((1801.591345240361, 866.1990171553026, 0.0), (1801.6187635862698, 865.8427607047952, 0.0))], [7, 667, 481, 703, 512, None, (1801.6139178025333, 867.4321370039261, 0.0), ((1801.6098542236446, 867.4841990073954, 0.0), (1801.617981381422, 867.3800750004567, 0.0))], [8, 511, 457, 603, 538, None, (1801.5712381097267, 867.797315379675, 0.0), ((1801.5609812723276, 867.9289471877333, 0.0), (1801.5814949471257, 867.6656835716167, 0.0))], [9, 829, 478, 858, 499, None, (1801.768005389129, 866.9753592634866, 0.0), ((1801.764549671694, 867.0190211505278, 0.0), (1801.7714611065637, 866.9316973764452, 0.0))], [10, 1032, 472, 1136, 527, None, (1801.6616424343174, 866.2723484163274, 0.0), ((1801.6498479783318, 866.4242220177541, 0.0), (1801.673436890303, 866.1204748149007, 0.0))], [11, 652, 478, 692, 513, None, (1801.625129787313, 867.4720998074624, 0.0), ((1801.6205926044395, 867.5301306334491, 0.0), (1801.6296669701865, 867.4140689814757, 0.0))], [12, 523, 456, 607, 533, None, (1801.6150544276693, 867.7851696635521, 0.0), ((1801.6055279935438, 867.9068046642283, 0.0), (1801.624580861795, 867.6635346628758, 0.0))], [13, 776, 475, 800, 500, None, (1801.770028826409, 867.1433683802486, 0.0), ((1801.7671611048174, 867.1795363232189, 0.0), (1801.7728965480007, 867.1072004372783, 0.0))]]
target = [[0, 728, 463, 785, 509, 'lanegroup',
           np.array([list(zip([632.5673278607485, 595.0064038929324, 723.9896110386784, 739.2077568337414, 756.3507025947408, 814.9641409917908, 770.6435112678597, 632.5673278607485],
                            [528.9128096702734, 528.3205624725372, 501.6543160656389, 498.422248120345, 498.66987355238507, 499.49252554956377, 530.8904272699301, 528.9128096702734]))]).reshape(-1,1,2)]]

skipping_info_0655=[[1, 'exit_view', ((0, 461), (176, 650))], [2, 'car_exit_segment', ((767, 473), (799, 492))], [10, 'ego_exit_segment', None], [11, 'car_exit_segment', ((766, 480), (801, 499))], [12, 'car_exit_segment', ((525, 469), (596, 530))], [14, 'car_exit_segment', ((517, 469), (592, 532))], [19, 'car_exit_segment', ((425, 477), (528, 548))], [20, 'car_exit_segment', ((200, 482), (381, 591))], [21, 'car_exit_segment', ((158, 480), (361, 595))], [22, 'car_exit_segment', ((114, 481), (341, 602))], [23, 'car_exit_segment', ((83, 478), (329, 606))], [24, 'car_exit_segment', ((14, 476), (310, 613))], [26, 'car_exit_segment', ((418, 460), (525, 541))], [29, 'car_exit_segment', ((395, 460), (513, 544))], [35, 'ego_exit_segment', None], [37, 'car_exit_segment', ((776, 475), (800, 500))], [38, 'car_exit_segment', ((0, 486), (218, 648))], [43, 'car_exit_segment', ((152, 467), (380, 599))], [44, 'car_exit_segment', ((3, 468), (287, 633))], [47, 'car_exit_segment', ((0, 467), (258, 641))], [48, 'car_exit_segment', ((771, 478), (806, 501))], [53, 'car_exit_segment', ((89, 461), (379, 614))], [55, 'car_exit_segment', ((252, 435), (482, 586))], [57, 'car_exit_segment', ((196, 430), (467, 598))], [65, 'meet_up', ((1, 446), (143, 645))], [115, 'car_exit_segment', ((406, 467), (561, 559))], [119, 'car_exit_segment', ((367, 464), (547, 565))], [123, 'car_exit_segment', ((306, 461), (520, 576))], [126, 'car_exit_segment', ((233, 476), (493, 606))], [127, 'car_exit_segment', ((1137, 478), (1196, 507))], [131, 'car_exit_segment', ((131, 477), (450, 631))], [132, 'ego_exit_segment', None], [135, 'car_exit_segment', ((0, 473), (392, 662))], [138, 'car_exit_segment', ((1, 481), (328, 710))], [139, 'car_exit_segment', ((684, 470), (764, 531))], [147, 'meet_up', ((0, 500), (221, 726))], [148, None, None], [149, None, None], [151, 'car_exit_segment', ((1353, 475), (1435, 505))], [181, 'meet_up', ((474, 439), (655, 560))], [182, 'ego_exit_segment', None], [203, 'car_exit_segment', ((774, 474), (805, 492))], [236, 'meet_up', ((728, 463), (785, 509))]]

def investigation_skipping(video_filename, skipping_info):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vid_name = f"./outputs/{video_filename}_skipping_investigation_new.mp4"
    display_video = cv2.VideoWriter(vid_name, fourcc, 1, (1600, 900))
    video_path = os.path.join(DATA_DIR, "videos", video_filename+'.mp4')
    vidcap = cv2.VideoCapture(video_path)
    frame_idx = 0
    current_frame_num, current_action, current_target_bbox = skipping_info[0]
    current_skipping_info_idx = 0
    current_bbox_appearance = 0
    while(vidcap.isOpened()):
        success, image = vidcap.read()
        if not success:
            break
        next_skipping_info_idx = current_skipping_info_idx + 1
        if next_skipping_info_idx < len(skipping_info):
            next_frame_num, next_action, next_target_bbox = skipping_info[next_skipping_info_idx]
        if frame_idx < next_frame_num:
            if current_action:
                image = cv2.putText(image, current_action, (50, 50),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            if current_target_bbox and current_bbox_appearance == 0:
                x, y = current_target_bbox[0]
                x_w, y_h = current_target_bbox[1]
                cv2.rectangle(image, (x, y), (x_w, y_h), (0,255,0), 4)
                current_bbox_appearance += 1
        if frame_idx == 207:
            for e in target:
                did, x, y, x_w, y_h, t, pts1 = e
                cv2.putText(image, f'obj_{did}', (x-20, y_h+10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.putText(image, t, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.rectangle(image, (x,y), (x_w, y_h), (0,255,0), 2)
                cv2.polylines(image, np.int32([pts1]), isClosed=True, color=(255,0,0), thickness = 2)
            cv2.imwrite('./outputs/0655_frame_207_skipping_investigation_new.jpg', image)
        display_video.write(image)
        frame_idx += 1
        if frame_idx == next_frame_num:
            current_skipping_info_idx = next_skipping_info_idx
            current_action = next_action
            current_target_bbox = next_target_bbox
            current_bbox_appearance = 0
    display_video.release()
investigation_skipping('boston-seaport-scene-0655-CAM_FRONT', skipping_info_0655)

In [6]:
optimized_query_start = time.time()
id_time_camId_filename = world.get_id_time_camId_filename(1)
print(f'optimized query time: {time.time()-optimized_query_start}')

execute: filter
execute: get_id_time_camId_filename
get_id_time_camId_filename 
        SELECT t0.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as t0
        
        JOIN Cameras USING (cameraId)
        WHERE ((t0.objectType LIKE 'car' OR t0.objectType LIKE 'truck') AND angleBetween(facingRelative(egoHeading,roadDirection(egoTranslation,egoHeading)),-15,15) AND (ST_Distance(egoTranslation,valueAtTimestamp(t0.translations,timestamp))<50) AND (EXISTS(SELECT 1
            FROM SegmentPolygon
            WHERE
                SegmentPolygon.__RoadType__intersection__ AND
                ST_Covers(SegmentPolygon.elementPolygon, valueAtTimestamp(t0.translations,timestamp))
        )) AND angleBetween(facingRelative((headingAtTimestamp(t0.itemHeadings,timestamp))::real,egoHeading),135,225) AND (minDistance(egoTranslation,'intersection')<10))
        
done execute node
Result length: 10
optimized query time: 72.545421600341

In [7]:
optimized_results = [e[3] for e in id_time_camId_filename]
construct_video(optimized_results, vid_name="modified.mp4")

In [ ]:
preprocess(world, DATA_DIR, ['scene-0757-CAM_FRONT'])

In [ ]:
base_query_start = time.time()
id_time_camId_filename_base = world.get_id_time_camId_filename(2)
print(f'based query time: {time.time()-base_query_start}')

In [ ]:
id_time_camId_filename_base

In [ ]:
base_results = [e[4] for e in id_time_camId_filename_base]
construct_video(base_results, base=True)

In [ ]:
false_negative = [f for f in base_results if f not in optimized_results]
print(len(false_negative))
construct_video(false_negative, vid_name="false_negative.avi")

In [ ]:
false_positive = [f for f in optimized_results if f not in base_results]
print(len(false_positive))
construct_video(false_positive, vid_name="false_positive.avi")